# Reverse engineer molecules

#### Apr 2024 version

## Enumerate signatures and molecules from Morgan fingerprints

In [43]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"

# packages
import numpy as np
import os

import pandas as pd
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

os.chdir(path_directory)
os.environ["RD_CANON"] = "True"

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import Chem
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

# Reverse engineer molecules from Morgan
from src.signature.enumerate_signature import enumerate_molecule_from_signature, enumerate_signature_from_morgan
from src.signature.enumerate_utils import reduced_fingerprint
from src.signature.Signature import MoleculeSignature
from src.signature.signature_old import signature_neighbor
from src.signature.signature_alphabet import (
    load_alphabet,
    morgan_vector_from_signature,
    SignatureAlphabet,
    signature_from_smiles,
)
from src.signature.utils import read_csv

In [44]:
# smarts or smiles
use_smarts = False

In [45]:
# paths
file_smiles = "./datasets/MetaNetX_weight_500_radius_3_size_1000"
if use_smarts:
    file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_nBits_2048_smarts"
else:
    file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048"

file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_boundary_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_full_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048"

# Load Smiles file
H, D = read_csv(file_smiles)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 0])))
print(f"Number of smiles: {len(Smiles)}")

# Get alphabet
Alphabet = load_alphabet(file_alphabet)
Alphabet.print_out()

['SMILES', 'SIG', 'SIG-NBIT'] 1000
Number of smiles: 1000
filename: ./datasets/MetaNetX_weight_500_r2_smiles_boundary_Alphabet_radius_2_nBits_2048.npz
radius: 2
nBits: 2048
splitcomponent: False
isomericSmiles: False
formalCharge: True
atomMapping: False
kekuleSmiles: False
allHsExplicit: False
maxvalence: 4
alphabet length: 183357


In [46]:
# fonctions test
def test_sol_ECFP(smis):
    fpgen = AllChem.GetMorganGenerator(radius=Alphabet.radius, fpSize=2048)
    ecfp_list = []
    for smi in smis:
        mol = Chem.MolFromSmiles(smi)
        morgan = fpgen.GetCountFingerprint(mol).ToList()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_ECFP_reduced(smis):
    ecfp_list = []
    for smi in smis:
        morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False).tolist()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_sig(smis):
    sig_list = []
    for smi in smis:
        sig = signature_from_smiles(smi, Alphabet, neighbor=True)[0]
        sign = signature_neighbor(sig)
        sig_list.append(sign)
    return len(set(tuple(i) for i in sig_list)) == 1

- ECFP reduced => Signature

In [47]:
list_i = []
list_smi = []
list_nsig = []
list_foundsig = []
list_ct = []
list_ct_solve = []
list_timeout = []

In [15]:
Alphabet.nBits = 2048
max_nbr_partition = int(1e5)

for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    st = time.time()

    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ft = time.time() - st

    print(f"{i} | {smi} | {len(Ssig)} | {int(foundsig)} | {ft:.4f} | {ct_solve:.4f}| {bool_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_nsig.append(len(Ssig))
    list_foundsig.append(foundsig)
    list_ct.append(ft)
    list_ct_solve.append(ct_solve)
    list_timeout.append(bool_timeout)

0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 1 | 1 | 2.2828 | 0.0059| False
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 1 | 1 | 2.0789 | 0.0039| False
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 1 | 1 | 2.4400 | 0.0030| False
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C | 1 | 1 | 2.8967 | 0.0040| False
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N | 1 | 1 | 1.8537 | 0.0010| False


In [17]:
100 * sum(list_foundsig) / len(list_foundsig)

100.0

In [18]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "Nsig": list_nsig,
        "FoundSig": list_foundsig,
        "CT enum ecfp_red to sig": list_ct,
        "CT solve by partitions": list_ct_solve,
        "Partitions timeout": list_timeout,
    }
)

df

# df.to_excel("RevSig_ecfp_to_sig", index=False)

ID                                                smi  Nsig  FoundSig  \
0   0            CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O     1      True   
1   1          OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O     1      True   
2   2               O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O     1      True   
3   3  CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...     1      True   
4   4                     COC(=O)c1sc(=S)n(-c2ccccc2)c1N     1      True   

   CT enum ecfp_red to sig  CT solve by partitions  Partitions timeout  
0                 2.282799                0.005888               False  
1                 2.078908                0.003913               False  
2                 2.440003                0.003002               False  
3                 2.896723                0.003992               False  
4                 1.853659                0.001012               False

- Signature => Smiles

In [29]:
list_i = []
list_smi = []
list_wt = []
list_nmol = []
list_foundmol = []
list_ct_mol = []
list_recursion_timeout = []

In [38]:
Alphabet.nBits = 0
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout")
for i in range(1):
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))
    
    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)

    st = time.time()
    smol, recursion_timeout = enumerate_molecule_from_signature(
        sig,
        Alphabet,
        smi,
        use_smarts=use_smarts,
        max_nbr_recursion=max_nbr_recursion,
        max_nbr_solution=max_nbr_solution,
        repeat=repeat,
        verbose=False,
    )

    ft = time.time() - st
    foundmol = smi in smol
    if foundmol == False:
        print(smol)

    print(f"{i} | {smi} | {wt:.1f} | {len(smol)} | {int(foundmol)} | {ft:.4f} | {recursion_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nmol.append(len(smol))
    list_foundmol.append(int(foundmol))
    list_ct_mol.append(ft)
    list_recursion_timeout.append(recursion_timeout)

ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
*-C(-*)-[CH3:1].SINGLE|*-N(-*)-[CH:1](-C)-O *-C(-*)-[OH:1].SINGLE|*-N(-*)-[CH:1](-C)-O *-C(-*)-[OH:1].SINGLE|*-N(-*)-[CH:1](-O)-C=* *-C(-*)=[CH:1]-N=*.DOUBLE|*-C=[C:1](-C)-C(-*)=*.SINGLE|*-C=[N:1]-C=* *-C(-*)=[O:1].DOUBLE|*-C(=*)-[C:1](=O)-C(-*)=* *-C(-*)=[O:1].DOUBLE|*-C(=*)-[C:1](=O)-C(-*)=* *-C(-*)=[O:1].DOUBLE|*-C(=*)-[C:1](=O)-O *-C(=*)-[C:1](=O)-C(-*)=*.DOUBLE|*-C(-*)=[O:1].SINGLE|*-C(=*)-[C:1](=O)-C(-*)=*.SINGLE|*-C(=*)-[C:1](=O)-O *-C(=*)-[C:1](=O)-C(-*)=*.DOUBLE|*-C(-*)=[O:1].SINGLE|*-C(=*)-[C:1](=O)-C(-*)=*.SINGLE|*-C=[C:1](-C)-C(-*)=* *-C(=*)-[C:1](=O)-O.DOUBLE|*-C(-*)=[O:1].SINGLE|*-C(=*)-[C:1](=O)-C(-*)=*.SINGLE|*-C(=*)-[OH:1] *-C(=*)-[CH3:1].SINGLE|*-C=[C:1](-C)-C(-*)=* *-C(=*)-[OH:1].SINGLE|*-C(=*)-[C:1](=O)-O *-C-[CH3:1].SINGLE|*-N(-*)-[CH2:1]-C *-C-[N:1](-C(-*)-*)-C(-*)-*.SINGLE|*-N(-*)-[CH2:1]-C.SINGLE|*-N(-*)-[CH:1](-C)-O.SINGLE|*-N(-*)-[CH:1](-O)-C=* *-C=[C:1](-C)-C(

In [21]:
100 * sum(list_foundmol) / len(list_foundmol)

100.0

In [22]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "weight": list_wt,
        "Nmol": list_nmol,
        "FoundMol": list_foundmol,
        "CT enum sig to mol": list_ct_mol,
        "Recursion timeout": list_recursion_timeout,
    }
)

df

# df.to_excel("RevSig_sig_to_mol", index=False)

ID                                                smi      weight  Nmol  \
0   0            CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O  286.116486     1   
1   1          OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O  345.969705     1   
2   2               O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O  322.131742     1   
3   3  CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...  420.272318     2   
4   4                     COC(=O)c1sc(=S)n(-c2ccccc2)c1N  266.018370     1   

   FoundMol  CT enum sig to mol  Recursion timeout  
0         1            0.029909              False  
1         1            0.035001              False  
2         1            0.131994              False  
3         1            0.147995              False  
4         1            0.029001              False

- ECFP reduced => Signature => Smiles

In [24]:
list_i = []
list_smi = []
list_wt = []
list_nsig = []
list_nsigtrue = []
list_foundsig = []
list_nmol = []
list_foundmol = []
list_ct_sig = []
list_ct_mol = []
list_ct_all = []
list_ct_solve = []

In [31]:
max_nbr_partition = int(1e5)
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time")
for i in range(5):
    # preparation
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=use_smarts, nbits=False, boundary_bonds=True, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    # ecfp_red => signature(s)
    st_1 = time.time()
    Alphabet.nBits = 2048
    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ct_sig = time.time() - st_1
    print(f"...enumeratesignature: nbr signatures {len(Ssig)} CPU-time {ct_sig:.4f}")

    # signature(s) => molecule(s)
    st_2 = time.time()
    Smol, Nsig = set(), 0
    Alphabet.nBits = 0
    for j in range(len(Ssig)):
        smol, recursion_timeout = enumerate_molecule_from_signature(
            Ssig[j],
            Alphabet,
            smi,
            use_smarts=use_smarts,
            max_nbr_recursion=max_nbr_recursion,
            max_nbr_solution=max_nbr_solution,
            repeat=repeat,
            verbose=False,
        )
        if len(smol):
            Nsig += 1
            print(f"...enumeratemolecule:  signature {j} nbr molecules {len(smol)}")
        Smol = Smol | set(smol)
    ct_mol = time.time() - st_2

    ct_total = time.time() - st_1
                
    Smolfinal = []
    for smi2 in Smol:
        if test_sol_ECFP_reduced([smi, smi2]):
            Smolfinal.append(smi2)
    if len(Smol) != len(Smolfinal):
        print("Smol", len(Smol), "Smolfinal", len(Smolfinal))

    foundmol = smi in Smolfinal
    if foundmol == False:
        print(Smolfinal)
        
    print(
        f"{i} | {smi} | {wt:.1f} | {len(Ssig)} | {Nsig} | {int(foundsig)} | {len(Smolfinal)} | {int(foundmol)} | {ct_total:.4f} | {ct_solve:.4f}"
    )
    
    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nsig.append(len(Ssig))
    list_nsigtrue.append(Nsig)
    list_foundsig.append(int(foundsig))
    list_nmol.append(len(Smolfinal))
    list_foundmol.append(int(foundmol))
    list_ct_sig.append(ct_sig)
    list_ct_mol.append(ct_mol)
    list_ct_all.append(ct_total)
    list_ct_solve.append(ct_solve)

ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
...enumeratesignature: nbr signatures 1 CPU-time 2.2290
*-C(-*)-C.SINGLE|*-N(-*)-C(-C)-O *-C(-*)-O.SINGLE|*-N(-*)-C(-C)-O *-C(-*)-O.SINGLE|*-N(-*)-C(-O)-C=* *-C(-*)=C-N=*.DOUBLE|*-C=C(-C)-C(-*)=*.SINGLE|*-C=N-C=* *-C(-*)=O.DOUBLE|*-C(=*)-C(=O)-C(-*)=* *-C(-*)=O.DOUBLE|*-C(=*)-C(=O)-C(-*)=* *-C(-*)=O.DOUBLE|*-C(=*)-C(=O)-O *-C(=*)-C(=O)-C(-*)=*.DOUBLE|*-C(-*)=O.SINGLE|*-C(=*)-C(=O)-C(-*)=*.SINGLE|*-C(=*)-C(=O)-O *-C(=*)-C(=O)-C(-*)=*.DOUBLE|*-C(-*)=O.SINGLE|*-C(=*)-C(=O)-C(-*)=*.SINGLE|*-C=C(-C)-C(-*)=* *-C(=*)-C(=O)-O.DOUBLE|*-C(-*)=O.SINGLE|*-C(=*)-C(=O)-C(-*)=*.SINGLE|*-C(=*)-O *-C(=*)-C.SINGLE|*-C=C(-C)-C(-*)=* *-C(=*)-O.SINGLE|*-C(=*)-C(=O)-O *-C-C.SINGLE|*-N(-*)-C-C *-C-N(-C(-*)-*)-C(-*)-*.SINGLE|*-N(-*)-C(-C)-O.SINGLE|*-N(-*)-C(-O)-C=*.SINGLE|*-N(-*)-C-C *-C=C(-C)-C(-*)=*.DOUBLE|*-C(-*)=C-N=*.SINGLE|*-C(=*)-C.SINGLE|*-C(=*)-C(=O)-C(-*)=* *-C=N-C=*.DOUBLE|*-N=C-C

OverflowError: can't convert negative value to unsigned int

In [ ]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "wt": list_wt,
        "Nsig": list_nsig,
        "NsigTrue": list_nsigtrue,
        "FoundSig": list_foundsig,
        "Nmol": list_nmol,
        "Foundmol": list_foundmol,
        "CT ecfp_sig": list_ct_sig,
        "CT sig_mol": list_ct_mol,
        "CT ecfp_mol": list_ct_all,
        "CT solve_partitions": list_ct_solve,
    }
)

df

# df.to_excel("RevSig_ecfp_to_mol.xlsx", index=False)